# 🚀 Fast Car Parts Training - Direct Folder Version

**No ZIP needed!** Upload the folder directly to Google Drive.

## Setup
1. Upload `car parts 50` folder to Google Drive
2. Enable T4 GPU in Colab
3. Run all cells

In [ ]:
import tensorflow as tf
import json, os

# Enable optimizations
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
tf.config.optimizer.set_jit(True)

print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# TODO: Update this to your folder path
# Example: '/content/drive/MyDrive/car parts 50'
DATASET_DIR = '/content/drive/MyDrive/car parts 50'

if os.path.exists(DATASET_DIR):
    print(f"✓ Found: {DATASET_DIR}")
    print(f"  Folders: {os.listdir(DATASET_DIR)}")
else:
    print(f"❌ Not found: {DATASET_DIR}")
    print("\nPlease upload 'car parts 50' folder to Google Drive")
    print("Then update DATASET_DIR path above")

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

augment = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

train_ds = tf.keras.utils.image_dataset_from_directory(
    f'{DATASET_DIR}/train', image_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE, shuffle=True, seed=42
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    f'{DATASET_DIR}/valid', image_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE, shuffle=False
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    f'{DATASET_DIR}/test', image_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE, shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"✓ {num_classes} classes: {class_names[:5]}...")

norm = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x,y: (augment(x), y), AUTOTUNE).map(lambda x,y: (norm(x), y), AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds.map(lambda x,y: (norm(x), y), AUTOTUNE).prefetch(AUTOTUNE)
test_ds = test_ds.map(lambda x,y: (norm(x), y), AUTOTUNE).prefetch(AUTOTUNE)

In [ ]:
base = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base.trainable = False

model = tf.keras.Sequential([
    base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax', dtype='float32')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5')])
print("✓ Model ready")

In [ ]:
print("Phase 1: Training (15-20 mins)...")
h1 = model.fit(train_ds, epochs=20, validation_data=val_ds,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('p1.h5', monitor='val_accuracy', save_best_only=True),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
    ])
print(f"✓ Phase 1: {max(h1.history['val_accuracy'])*100:.2f}%")

In [ ]:
print("Phase 2: Fine-tuning (10-15 mins)...")
base.trainable = True
for layer in base.layers[:100]: layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5')])

h2 = model.fit(train_ds, epochs=10, validation_data=val_ds,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('final.h5', monitor='val_accuracy', save_best_only=True),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ])
print(f"✓ Phase 2: {max(h2.history['val_accuracy'])*100:.2f}%")

In [ ]:
results = model.evaluate(test_ds)
print(f"\n🎉 Test Accuracy: {results[1]*100:.2f}%")
print(f"🎉 Top-5 Accuracy: {results[2]*100:.2f}%")

model.save('visual_search_model.h5')
with open('class_labels.json', 'w') as f:
    json.dump({i: name for i, name in enumerate(class_names)}, f, indent=2)
print("\n✓ Saved!")

In [ ]:
from google.colab import files
files.download('visual_search_model.h5')
files.download('class_labels.json')
print("✓ Downloaded! Copy to ai-module/models/")